In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import re
import os
import unicodedata
import numpy as np
import torch.optim as optim
from torchtext.datasets import TranslationDataset,Multi30k
from torchtext.data import Field,BucketIterator
device=torch.device('cuda:0')
import spacy
import random
import math
import time

In [ ]:
SEED=1024

random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic=True

In [ ]:
spacy_de=spacy.load('de')
spacy_en=spacy.load('en')
def token_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)][::-1]
def token_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]
SRC=Field(tokenize=token_de,init_token='<sos>',eos_token='<eos>',lower=True)
TRG=Field(tokenize=token_en,init_token='<sos>',eos_token='<eos>',lower=True)
train_data,valid_data,test_data=Multi30k.splits(exts=('.de','.en'),fields=(SRC,TRG))

print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")
print(vars(train_data.examples[0]))

In [ ]:
SRC.build_vocab(train_data,min_freq=2)
TRG.build_vocab(train_data,min_freq=2)
print(len(SRC.vocab))
print(len(TRG.vocab))
BATCH_SIZE=128
train_iter,valid_iter,test_iter=BucketIterator.splits((train_data,valid_data,test_data),batch_size=BATCH_SIZE,device=device)
print(train_iter)

In [ ]:
class Encoder(nn.Module):
    def __init__(self,input_dim,emb_dim,hid_dim,n_layers,dropout):
        super().__init__()
        
        self.input_dim=input_dim
        self.emb_dim=emb_dim
        self.hid_dim=hid_dim
        self.n_layers=n_layers
        self.dropout=dropout
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self,src):
        embedded=self.dropout(self.embedding(src))
        
        outputs,(hidden,cell)=self.rnn(embedded)
        
        return hidden,cell
class Decoder(nn.Module):
    def __init__(self,output_dim,emb_dim,hid_dim,n_layers,dropout):
        super().__init__()
        
        self.emb_dim=emb_dim
        self.hid_dim=hid_dim
        self.output_dim=output_dim
        self.n_layers=n_layers
        self.dropout=dropout
        
        self.embedding=nn.Embedding(output_dim,emb_dim)
        
        self.rnn=nn.LSTM(emb_dim,hid_dim,n_layers,dropout=dropout)
        
        self.out=nn.Linear(hid_dim,output_dim)
        
        self.dropout=nn.Dropout(dropout)
    def forward(self,inputs,hidden,cell):
        inputs=inputs.unsqueeze(0)
        embedded=self.dropout(self.embedding(inputs))
        
        output,(hidden,cell)=self.rnn(embedded,(hidden,cell))
        
        prediction=self.out(output.squeeze(0))
        
        return prediction,hidden,cell
class Seq2Seq(nn.Module):
    def __init__(self,encoder,decoder,device):
        super().__init__()
        
        self.encoder=encoder
        self.decoder=decoder
        self.device=device
        
        assert encoder.hid_dim==decoder.hid_dim,\
            "编码层和解码层隐藏层必须相同"
        assert encoder.n_layers==decoder.n_layers,\
            "编码器解码器层数必须相同"
    def forward(self,src,trg,teacher_forcing_ratio=0.5):
        
        batch_size=trg.shape[1]
        max_len=trg.shape[0]
        
        trg_vacab_size=self.decoder.output_dim
        
        outputs=torch.zeros(max_len,batch_size,trg_vacab_size).to(self.device)
        
        hidden,cell=self.encoder(src)
        
        inputs=trg[0,:]
        
        for t in range(1,max_len):
            output,hidden,cell=self.decoder(inputs,hidden,cell)
            outputs[t]=output
            teacher_forcing=random.random()<teacher_forcing_ratio
            top1=output.max(1)[1]
            inputs=(trg[t] if teacher_forcing else top1)
            
        return outputs